In [1]:
import time

from bokeh.plotting import figure, show, Session, cursession, output_server

from data.datahandler import yield_from_last
from data.datastream import queryinput_to_csv_yf, split_csvline_yf, queryinput_to_gen_yf

In [2]:
session = Session(load_from_config=False, root_url = "http://127.0.0.1:5006/")
output_server("scatter", url = "http://127.0.0.1:5006/")

Using saved session configuration for http://127.0.0.1:5006/
To override, pass 'load_from_config=False' to Session


In [3]:
class ScatterDeltas(object):
        
    #inizializza dplot. Le variabili del plot non inizializzate (titolo, etc.) sono comunque accessibili dall'esterno
    def __init__(self): #step tra due dati in gg
        #inizializza plot
        self.plot = figure(
            plot_width = 600,
            plot_height = 600,
        )
        self.plot.circle(
            x = [],
            y = [],
            size = 6,
            color = "navy",
            alpha = 0.5,
            name = "scatter"
        )
        self.ds = self.plot.select({"name":"scatter"})[0].data_source
        #in questa variabile viene salvato il dato utilizzato per calcolare il delta relativo al tick temporale precedente
        self.x_olddata = None 
        self.y_olddata = None
        
    #ritorna l'oggetto Plot    
    def get_plot(self):
        return self.plot

    #IN:lista (wrap effettuato per rendere la sintassi del metodo uguale per i vari chart) in cui ogni elemento è:
    #A) una linea del csv risultante da una query a YF, se dato valido. 
    #B) None, se simulato arrivo dato invalido o assenza dati
    #Nel caso specifico due linee:
    #indice 0: linea del csv relativa a titolo x; indice 1: linea del csv relativa a titolo y
    #OUT: lista dei valori da passare al metodo update() per aggiornare il grafico
    #A) indice 0: valore per calcolo delta titolo x, indice 1: valore per calcolo delta titolo y; se dato valido
    #B) None, se simulato arrivo dato invalido o assenza dati
    def csvlines_to_data_yf(self, csvlines):
        data = [None, None]
        for i in range(2):
            if csvlines[i] != None: data[i] = split_csvline_yf(csvlines[i], "byte")[4] #utilizza close
            else: pass
        return data
    
    #CLASSE NON SPECIFICA PER DATI YF. NON IMPLEMENTA CONTROLLO SULLA DATA DEGLI INPUT, ASSUMENDO CHE ESSA SIA UGUALE.
    #TALE CONTROLLO, SE NECESSARIO, VA EFFETTUATO DALLA APPLICAZIONE CHE PASSA I DATI
    def update(self, data):
        if data[0] == None or data[1] == None: #caso in cui si ha assenza di dati di almeno un titolo
        #suppone che, se per un titolo un giorno non ci sono dati, al posto della linea da esso attesa riceve None
        #Se riceve None, cancella la memoria dei dati dei titoli x e y relativi al tick temporale precedente (per 
        #calcolare un delta valido servono due dati validi consecutivi sia del titolo x che del titolo y)
            self.x_olddata = None
            self.y_olddata = None
        else: #casi in cui entrambi i dati x e y ricevuti al corrente tick temporale sono validi
        #se è la prima volta che riceve dei dati non appende alcun dato al ds del plot (non si può calcolare un delta)
        #ma semplicemente salva tali dati per calcolare il delta alla ricezione dei successivi dati (se validi)
            x_data = data[0]
            y_data = data[1]
            if self.x_olddata != None: #caso in cui anche i dati x e y ricevuti al tick temporale precedente sono validi
            #Appende al ds del plot un punto con ascissa pari al delta tra il valore del titolo x ricevuto a questo tick
            #temporale e al tick temporale precedente, e ordinata calcolata analogamente con i dati del titolo y
                self.ds.data["x"].append(x_data-self.x_olddata)
                self.ds.data["y"].append(y_data-self.y_olddata)
            #dati ricevuti memorizzati come relativi al tick temporale precedente alla prossima chiamata
            self.x_olddata = x_data
            self.y_olddata = y_data
            #ritorna il ds del plot aggiornato con ascissa e ordinata del punto da plottare calcolate dai dati forniti
        return [self.ds] #wrap in lista, per poter effettuare l'estrazione degli oggetti aggiornati nel modo più generico

In [4]:
#Inizializzazione variabili query
#GOOGL, ENEL.MI, UCG.MI, ISP.MI
x_quote_id = "UCG.MI"
y_quote_id = "ISP.MI"
start_y = 2014
start_m = 1
start_d = 1
stop_y = 2015
stop_m = 6
stop_d = 30
step = "d"

In [ ]:
x_datagen = queryinput_to_gen_yf(x_quote_id, start_y, start_m, start_d, stop_y, stop_m, stop_d, step)
y_datagen = queryinput_to_gen_yf(y_quote_id, start_y, start_m, start_d, stop_y, stop_m, stop_d, step)
s = ScatterDeltas()
show(s.get_plot())
while True:
    data = s.csvlines_to_data_yf([next(x_datagen), next(y_datagen)])
    cursession().store_objects(*s.update(data))
    time.sleep(0.2)

In [6]:
#TEST CLASSE ScatterDeltasYF

In [7]:
#simulo dati validi formattati come csv di YF
#validline1 differiscono solo per data 
validline1 = b'2014-01-03,1.000,2.000,3.000,4.000,1000000,5.000\n'
    #delta temporale giornaliero
validline1A = b'2014-01-04,1.000,2.000,3.000,4.000,1000000,5.000\n'
validline1B = b'2014-01-05,1.000,2.000,3.000,4.000,1000000,5.000\n'
    #delta temporale mensile
validline1C = b'2014-02-03,1.000,2.000,3.000,4.000,1000000,5.000\n'
validline1D = b'2014-03-03,1.000,2.000,3.000,4.000,1000000,5.000\n'
    #delta temporale annuale
validline1E = b'2015-01-03,1.000,2.000,3.000,4.000,1000000,5.000\n'
validline1F = b'2016-01-03,1.000,2.000,3.000,4.000,1000000,5.000\n'
#validline2 per data e valore
validline2 = b'2014-01-01,3.000,4.000,5.000,6.000,2000000,7.000\n'
validline2A = b'2014-01-04,5.000,4.000,3.000,2.000,3000000,1.000\n'
validline2B = b'2014-01-15,8.000,11.000,14.000, 16.000,1000000,18.000\n'
validline2C = b'2014-01-30,25.000,20.000,15.000,10.000,500000,5.000\n'
validline2D = b'2014-01-31,-1,-2,-3,-4,500000,-5\n'
invalidline = None

In [11]:
s = ScatterDeltas()
show(s.get_plot())
#Qui testa tutti i casi in cui il grafico non deve aggiornarsi
#testa arrivo None, None subito
cursession().store_objects(*s.update(s.csvlines_to_data_yf([invalidline, invalidline])))
#testa arrivo None e dato valido, sia come singolo ingresso, che come due consecutivi relativi allo stesso titolo
cursession().store_objects(*s.update(s.csvlines_to_data_yf([invalidline, validline1])))
cursession().store_objects(*s.update([invalidline, split_csvline_yf(validline2)[4]]))
#testa arrivo di singola coppia di dati validi, seguita da singola coppia di dati non valida (e il viceversa)
cursession().store_objects(*s.update(s.csvlines_to_data_yf([validline1, validline2])))
cursession().store_objects(*s.update(s.csvlines_to_data_yf([validline1, invalidline])))
cursession().store_objects(*s.update(s.csvlines_to_data_yf([validline1, validline2])))

[4.0, 6.0]


In [9]:
#qui testa la ricezione di dati validi consecutivi
cursession().store_objects(*s.update(s.csvlines_to_data_yf([validline1, validline2])))
#uguale al dato valido precedente --> out(0,0)
cursession().store_objects(*s.update(s.csvlines_to_data_yf([validline2, validline1])))
#uguale al dato valido precedente --> out(2,-2)
cursession().store_objects(*s.update(s.csvlines_to_data_yf([validline2, validline1])))
#uguale al dato valido precedente --> out(0,0)
#si vede come, plottando più risultati con le stesse coordinate, non si hanno problemi; 
#il punto viene plottato più scuro

In [14]:
#verifico corretto funzionamento di csvlines_to_data_yf
print(s.csvlines_to_data_yf([validline1, validline2]))
print(s.csvlines_to_data_yf([invalidline, validline2]))
print(s.csvlines_to_data_yf([validline1, invalidline]))
print(s.csvlines_to_data_yf([invalidline, invalidline]))

[4.0, 6.0]
[None, 6.0]
[4.0, None]
[None, None]
